In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [ ]:
local_llm = LLM(
    model="ollama/llama3.1",
    base_url="http://localhost:11434"
)

gemini_llm = LLM(
    model="gemini-2.5-flash",
    api_key=os.getenv("GEMINI_API_KEY")
)

openai_llm = LLM(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY")
)

In [3]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful support representative in your team",
	backstory="""
	You work at crewAI (https://crewai.com) and are now working on providing
	support to {customer}, a super important customer for your company.
	You need to make sure that you provide the best support!
	Make sure to provide full complete answers,and make no assumptions.
	""",
	allow_delegation=False,
    llm=gemini_llm,
	verbose=True
)

support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the best support quality assurance in your team",
	backstory="""
	You work at crewAI (https://crewai.com) and are now working with your team on a 
	request from {customer} ensuring that the support representative is providing the best support possible.
	You need to make sure that the support representative is providing full complete answers, 
	and make no assumptions.
	""",
	verbose=True,
	llm=gemini_llm
	# Notice no allow_delegation=False. This agent can delegate to the support agent.
)

In [4]:
search_tool = SerperDevTool()

scrape_tool = ScrapeWebsiteTool()

docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)


Tools can be assigned to an Agent or to a Task.
If tool is given to the agent, it belongs to the agent, the agent can use it for any task.
If tool is given to the task, it belongs to the task, the agent will only use it for that specific task.

Task tools override agent tools.


In [5]:
# Tasks

inquiry_resolution = Task(
    description="""
    {customer} just reached out with a super important ask:\n
    {inquiry}\n\n
    {person} from {customer} is the one that reached out. 
    Make sure to use everything you know to provide the best support possible.
    You must strive to provide a complete and accurate response to the customer's inquiry.
    """,
    expected_output="""
    A detailed, informative response to the customer's inquiry that addresses all aspects of their question.\n
    The response should include references to everything you used to find the answer, including external data or solutions. 
    Ensure the answer is complete, leaving no questions unanswered, and maintain a helpful and friendly tone throughout.
    """,
	tools=[docs_scrape_tool],
    agent=support_agent,
)


quality_assurance_review = Task(
    description="""
    Review the response drafted by the Senior Support Representative for {customer}'s inquiry. 
    Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards 
    expected for customer support.\n
    Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful 
    and friendly tone.\n
    Check for references and sources used to find the information, ensuring the response is well-supported 
    and leaves no questions unanswered.
    """,
    expected_output="""
    A final, detailed, and informative response ready to be sent to the customer.\n"
    This response should fully address the customer's inquiry, incorporating all relevant feedback and improvements.\n
	Don't be too formal, we are a chill and cool company but maintain a professional and friendly tone throughout.
    """,
    agent=support_quality_assurance_agent,
)


In [6]:
# Crew

# By default, CrewAI attempts to use OpenAI's embedding model for its memory/RAG features, 
# which requires the OPENAI_API_KEY environment variable.
# Even though you configured your agents to use Gemini (cloud_llm), 
# the memory system is separate and defaults to OpenAI.

crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    memory=True,
    verbose=True
)

In [7]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": """
        I need help with setting up a Creww and kicking it off, specifically, how can I add memory to my crew?
        Can you provide guidance? 
        """
}

In [ ]:
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9e453dd4-3f2e-47ec-9b94-77d319fdf708                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

ERROR:root:Error during short_term search: Error code: 429 - {'error': {'message': 'You exceeded your current 
quota, please check your plan and billing details. For more information on this error, read the docs: 
https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 
'code': 'insufficient_quota'}} in query.
Traceback (most recent call last):
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/crewai/memory/storage/rag_storage.py", line 
184, in search
    return client.search(
           ^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/crewai/rag/chromadb/client.py", line 432, in 
search
    results: QueryResult = collection.query(
                           ^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/Collection.py", line 213,
in query
    query_request = self._validate_and_prepare_query_request(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 95, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 316, in _validate_and_prepare_query_request
    request_embeddings = self._embed_record_set(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 557, in _embed_record_set
    return self._embed(
           ^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 572, in _embed
    return self._embedding_function.embed_query(input=input)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/types.py", line 701, in 
embed_query
    return self.__call__(input)
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/types.py", line 709, in __call__
    result = call(self, input)
             ^^^^^^^^^^^^^^^^^
  File 
"/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/utils/embedding_functions/openai_embedding_
function.py", line 127, in __call__
    response = self.client.embeddings.create(**embedding_params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/openai/resources/embeddings.py", line 132, in
create
    return self._post(
           ^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/openai/_base_client.py", line 1259, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/openai/_base_client.py", line 1047, in 
request
    raise self._make_status_error_from_response(err.response) from None
openai.RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your 
plan and billing details. For more information on this error, read the docs: 
https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 
'code': 'insufficient_quota'}} in query.

ERROR:root:Error during entities search: Error code: 429 - {'error': {'message': 'You exceeded your current quota, 
please check your plan and billing details. For more information on this error, read the docs: 
https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 
'code': 'insufficient_quota'}} in query.
Traceback (most recent call last):
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/crewai/memory/storage/rag_storage.py", line 
184, in search
    return client.search(
           ^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/crewai/rag/chromadb/client.py", line 432, in 
search
    results: QueryResult = collection.query(
                           ^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/Collection.py", line 213,
in query
    query_request = self._validate_and_prepare_query_request(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 95, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 316, in _validate_and_prepare_query_request
    request_embeddings = self._embed_record_set(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 557, in _embed_record_set
    return self._embed(
           ^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 572, in _embed
    return self._embedding_function.embed_query(input=input)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/types.py", line 701, in 
embed_query
    return self.__call__(input)
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/types.py", line 709, in __call__
    result = call(self, input)
             ^^^^^^^^^^^^^^^^^
  File 
"/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/utils/embedding_functions/openai_embedding_
function.py", line 127, in __call__
    response = self.client.embeddings.create(**embedding_params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/openai/resources/embeddings.py", line 132, in
create
    return self._post(
           ^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/openai/_base_client.py", line 1259, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/openai/_base_client.py", line 1047, in 
request
    raise self._make_status_error_from_response(err.response) from None
openai.RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your 
plan and billing details. For more information on this error, read the docs: 
https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 
'code': 'insufficient_quota'}} in query.

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - If possible, provide the AI with access to linked documentation sections (e.g., 'Agents' documentation)      │
│  when such a reference is found in the initial document. This would allow it to fully answer specific           │
│  follow-up questions like 'how to add memory' with concrete examples.                                           │
│  - For tasks involving highly specific technical details that are only mentioned as being in other              │
│  documentation, consider if a multi-step retrieval process would be beneficial to gather all necessary info...  │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 4023.31ms ───────────────────────────────────────────╯

...

ERROR:root:Error during entities save: Error code: 429 - {'error': {'message': 'You exceeded your current quota, 
please check your plan and billing details. For more information on this error, read the docs: 
https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 
'code': 'insufficient_quota'}} in upsert.
Traceback (most recent call last):
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/crewai/memory/storage/rag_storage.py", line 
162, in save
    client.add_documents(
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/crewai/rag/chromadb/client.py", line 328, in 
add_documents
    collection.upsert(
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/Collection.py", line 442,
in upsert
    upsert_request = self._validate_and_prepare_upsert_request(
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 95, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 424, in _validate_and_prepare_upsert_request
    upsert_embeddings = self._embed_record_set(record_set=upsert_records)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 557, in _embed_record_set
    return self._embed(
           ^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/models/CollectionCommon.py", 
line 574, in _embed
    return self._embedding_function(input=input)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/api/types.py", line 709, in __call__
    result = call(self, input)
             ^^^^^^^^^^^^^^^^^
  File 
"/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/chromadb/utils/embedding_functions/openai_embedding_
function.py", line 127, in __call__
    response = self.client.embeddings.create(**embedding_params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/openai/resources/embeddings.py", line 132, in
create
    return self._post(
           ^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/openai/_base_client.py", line 1259, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vinothhaldorai/miniforge3/lib/python3.12/site-packages/openai/_base_client.py", line 1047, in 
request
    raise self._make_status_error_from_response(err.response) from None
openai.RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your 
plan and billing details. For more information on this error, read the docs: 
https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 
'code': 'insufficient_quota'}} in upsert.

📋 Task: 98f27d32-0ea4-42ca-8440-5c7c7717172b
Assigned to: Support Quality Assurance Specialist
Status: ✅ Completed
└── 🧠 Memory Update Overall
    ├── ✅ Short Term Memory Memory Saved (1788.15ms)
    ├── ✅ Long Term Memory Memory Saved (16.66ms)
    └── ✅ Entity Memory Memory Saved (13173.58ms)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 98f27d32-0ea4-42ca-8440-5c7c7717172b                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9e453dd4-3f2e-47ec-9b94-77d319fdf708                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Hey Andrew!                                                                                      │
│                                                                                                                 │
│  It's absolutely fantastic to hear from DeepLearningAI! I'd be more than happy to help you with setting up      │
│  your Crew and understanding how to add memory to it. You've definitely come to the right place for support!    │
│                                                                                                                 │
│  Let's dive into setting up your Crew and integrating memory effectively.                                       │
│                                                                                                                 │
│  ### Setting Up Your Crew                                                                                       │
│                                                                                                                 │
│  You're on the right track with the documentation! The page                                                     │
│  `https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/` actually provides a great example for        │
│  assembling your agents and tasks into a Crew object. If you scroll down a bit, you'll find a clear code        │
│  snippet that illustrates exactly how to instantiate your `Crew`:                                               │
│                                                                                                                 │
│  ```python                                                                                                      │
│  crew = Crew(                                                                                                   │
│      agents=[researcher, writer],                                                                               │
│      tasks=[research_task, write_task],                                                                         │
│      verbose=True # You can set it to 1 or 2 for different levels of log                                        │
│  )                                                                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
│  This section shows you how to bring your pre-defined agents and tasks together to form your operational crew.  │
│                                                                                                                 │
│  For a more comprehensive, step-by-step walkthrough from the very beginning, including setting up your          │
│  environment and defining individual agents and tasks before bringing them into a Crew, I highly recommend      │
│  checking out our **Getting Started Guide**:                                                                    │
│  *   **[CrewAI Getting Started Guide](https://docs.crewai.com/how-to/Getting-Started/)**                        │
│                                                       

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯